In [36]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.stats import truncnorm
import time
import csv


def get_s_cond_t_params(player_1_sigma, player_2_sigma, player_1_mean, player_2_mean, t_i):
    beta = 3  # från uppgiften

    # Covariance calculations
    covariance_t_given_s = beta
    covariance_ss = np.array([[player_1_sigma, 0], [0, player_2_sigma]])
    A = np.array([1, -1]).reshape([1, 2])
    inv_covariance_ss = np.linalg.inv(covariance_ss)
    ACA = np.matmul(A.T, A) * (1 / covariance_t_given_s)
    covariance_s_cond_t = np.linalg.inv(inv_covariance_ss + ACA)

    # Mean calculations
    player_means = np.array([player_1_mean, player_2_mean]).reshape([2, 1])
    a = covariance_s_cond_t
    b = np.matmul(inv_covariance_ss, player_means)
    c = A.T * (1 / covariance_t_given_s) * t_i
    mean_s_cond_t = np.matmul(a, b + c)

    return mean_s_cond_t, covariance_s_cond_t


def P_s_cond_t(t_i, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma):
    # player_1_mean = 25; player_2_mean = 25; player_1_sigma = 8.3**2; player_2_sigma = 8.3**2

    mean_s_cond_t, cov_s_cond_t = get_s_cond_t_params(player_1_mean=player_1_mean, player_1_sigma=player_1_sigma,
                                                      player_2_mean=player_2_mean, player_2_sigma=player_2_sigma,
                                                      t_i=t_i)

    return np.random.multivariate_normal(mean=mean_s_cond_t.reshape(2), cov=cov_s_cond_t,
                                         check_valid='warn', tol=1e-8)


def P_t_cond_s(s_i, t_game):
    s_diff = s_i[0] - s_i[1]
    beta = 3
    t_sigma = beta

    if t_game > 0:  # case for when y=1
        a, b = (0 - s_diff) / t_sigma, np.inf
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    elif t_game < 0:  # case for when y=-1
        a, b = -np.inf, (0 - s_diff) / t_sigma
        t = truncnorm.rvs(a, b) * t_sigma + s_diff
        return t
    else:
        print("ERROR, TIES PRESENTLY NOT ALLOWED")


def gibbs_sampler(L, player_1_stats, player_2_stats, t_game):
    player_1_mean, player_1_sigma = player_1_stats
    player_2_mean, player_2_sigma = player_2_stats
    s_i = [player_1_mean, player_2_mean]

    t_obs = np.zeros(L)
    s_obs = np.zeros([L, 2])

    for i in range(L):
        t_i_plus_1 = P_t_cond_s(s_i, t_game=t_game)
        s_i_plus_1 = P_s_cond_t(t_i_plus_1, player_1_mean, player_1_sigma, player_2_mean, player_2_sigma)

        t_obs[i] = t_i_plus_1
        s_obs[i, :] = s_i_plus_1

        s_i = s_i_plus_1
        # plt.scatter(s_obs[:, 0], s_obs[:, 1])
        # plt.pause(0.1)
    # plt.show()

    # plt.plot(s_obs[:, 0]); plt.plot(s_obs[:, 1]); plt.show()

    player_1_stats_estimate = [np.mean(s_obs[:, 0]), np.var(s_obs[:, 0])] # [mean, variance] of samples
    player_2_stats_estimate = [np.mean(s_obs[:, 1]), np.var(s_obs[:, 1])] # [mean, variance] of samples


    #plot_hist_normal(player_1_stats[0], player_1_stats[1], L, color='blue') # Normal curve for prior
    # Histogram and normal curve (scaled) for posterior
    #plot_hist_normal(player_1_stats_estimate[0], player_1_stats_estimate[1], L, color='blue', samples=s_obs[:, 0])
    #plot_hist_normal(player_2_stats_estimate[0], player_2_stats_estimate[1], L, color='green', samples=s_obs[:, 1])
    #plt.show()

    # print("s_1|y=1: mean =", player_1_stats_estimate[0], "; cov =", player_1_stats_estimate[1])
    # print("s_2|y=1: mean =", player_2_stats_estimate[0], "; cov =", player_2_stats_estimate[1])

    return player_1_stats_estimate, player_2_stats_estimate

# Plots histogram and fitted normal curve
def plot_hist_normal(mu, variance, prior_mu, prior_variance, **kwargs):
    samples = kwargs.get('samples', list)
    if kwargs:
        plt.hist(samples, bins=50, color="blue")
    sigma = np.sqrt(variance)
    x = np.linspace(mu - 3 * sigma, mu + 3 * sigma, 100)
    plt.plot(x, stats.norm.pdf(x, mu, sigma), color="blue")
    plt.plot(x, stats.norm.pdf(x, prior_mu, prior_variance**0.5), color="red")


def make_stats_dictionary(filename, stats_dictionary, printable):
    with open(filename) as csvfile:
        reader = csv.DictReader(csvfile)
        mean = 25; variance = (25 / 3) ** 2  # TrueSkill prior parameters before any games
        gamelist = []
        for row in reader:
            # Create dictionary (=map) with keyword 'team' and value [mean, variance]
            stats_dictionary[row['team1']] = [mean, variance]
            # Add teams and result to list
            gamelist.append([row['team1'], row['team2'], int(row['score1']) - int(row['score2'])])

            if printable == 1:  # Print the whole list
                print(f"{row['team1']} vs {row['team2']}: {row['score1']}-{row['score2']}")  # Access by column header
                      
    #stats_dictionary = random.shuffle(stats_dictionary, random)

    return stats_dictionary, gamelist


# Ranking by mean (should perhaps be improved to mean - 3 * sigma)
def ranking(stats_dictionary):
    sorted_teams = sorted(stats_dictionary.items(), key=lambda x: x[1], reverse=True)
    print("\nList of teams ranked by mean skill in descending order:\n")
    for i in sorted_teams:
        print(i[0], i[1])
                      


if __name__ == '__main__':
    # Make dictionary of team stats (mean and variance) & list of all games with result
    stats_dictionary, result_list = make_stats_dictionary('SerieA.csv', stats_dictionary={}, printable=0)
    print(stats_dictionary)
                      
    # If you want to randomize
    x = np.random.choice(range(len(result_list)), size=len(result_list), replace=False)
    randomized_list = []
    for i in x:
        randomized_list.append(result_list[i])
    result_list = randomized_list

    # Iterate game list using Gibbs-sampler to estimate posterior for skills, using them as new priors
    for i in range(len(result_list)):
        # result_list holds [team1, team2, result = score1 - score2]
        print(f"\nResult game {i}:   {result_list[i]}")
        team1 = result_list[i][0]
        team2 = result_list[i][1]
        result = result_list[i][2]
        # stats_dictionary with keyword 'teamname' and value [mean, variance]
        print(f"Stats before game: {stats_dictionary[team1]}, {stats_dictionary[team2]}")

        if result == 0:  # ignore tied games for now
            print("Game ignored due to tie")
            print(f"Stats after game:  {stats_dictionary[team1]}, {stats_dictionary[team2]}")

        else:
            player_1_stats_posterior, player_2_stats_posterior = gibbs_sampler(L=100,
                                                                               player_1_stats=stats_dictionary[team1],
                                                                               player_2_stats=stats_dictionary[team2],
                                                                               t_game=result)
            # Update team stats so posterior makes new prior
            stats_dictionary[team1] = player_1_stats_posterior
            stats_dictionary[team2] = player_2_stats_posterior
            print(f"Stats after game:  {player_1_stats_posterior}, {player_2_stats_posterior}")

    print(stats_dictionary)
    ranking(stats_dictionary)

{'Chievo': [25, 69.44444444444446], 'Lazio': [25, 69.44444444444446], 'Torino': [25, 69.44444444444446], 'Sassuolo': [25, 69.44444444444446], 'Parma': [25, 69.44444444444446], 'Empoli': [25, 69.44444444444446], 'Bologna': [25, 69.44444444444446], 'Atalanta': [25, 69.44444444444446], 'Juventus': [25, 69.44444444444446], 'Napoli': [25, 69.44444444444446], 'Spal': [25, 69.44444444444446], 'Udinese': [25, 69.44444444444446], 'Inter': [25, 69.44444444444446], 'Genoa': [25, 69.44444444444446], 'Frosinone': [25, 69.44444444444446], 'Fiorentina': [25, 69.44444444444446], 'Cagliari': [25, 69.44444444444446], 'Roma': [25, 69.44444444444446], 'Milan': [25, 69.44444444444446], 'Sampdoria': [25, 69.44444444444446]}

Result game 0:   ['Napoli', 'Roma', 0]
Stats before game: [25, 69.44444444444446], [25, 69.44444444444446]
Game ignored due to tie
Stats after game:  [25, 69.44444444444446], [25, 69.44444444444446]

Result game 1:   ['Sassuolo', 'Napoli', 0]
Stats before game: [25, 69.44444444444446], 

Stats after game:  [26.172320900028446, 22.625639195416355], [44.659255482494466, 43.42581404990044]

Result game 34:   ['Lazio', 'Atalanta', -2]
Stats before game: [29.83055105287698, 28.895866733736938], [30.822681941115608, 39.92588445295856]
Stats after game:  [27.241464372303064, 16.521033930764222], [33.818233709148274, 28.83273794614558]

Result game 35:   ['Roma', 'Bologna', 1]
Stats before game: [25, 69.44444444444446], [26.526551093959597, 42.062313186590885]
Stats after game:  [31.16173176546075, 35.34016612682422], [23.15982277564894, 24.063128028349052]

Result game 36:   ['Torino', 'Juventus', -1]
Stats before game: [26.706008094161476, 28.7135639300576], [44.659255482494466, 43.42581404990044]
Stats after game:  [25.81991576061168, 33.69190341137742], [46.27919104834187, 32.87385060470588]

Result game 37:   ['Chievo', 'Sassuolo', -2]
Stats before game: [21.197108079995793, 33.857276838406186], [24.352237831793605, 26.5218196526606]
Stats after game:  [18.935089432839607

Stats after game:  [26.524145674526295, 8.47463096195065], [23.88467573050439, 7.2951434200046466]

Result game 71:   ['Milan', 'Torino', 0]
Stats before game: [39.864333181784446, 27.129922489259435], [29.82095230840928, 31.0137625277885]
Game ignored due to tie
Stats after game:  [39.864333181784446, 27.129922489259435], [29.82095230840928, 31.0137625277885]

Result game 72:   ['Cagliari', 'Napoli', -1]
Stats before game: [29.581423684385154, 42.831791158517355], [34.3715795253511, 10.612568004195202]
Stats after game:  [22.72329089766916, 33.57326872051188], [36.016270441450416, 9.141034357062974]

Result game 73:   ['Frosinone', 'Sampdoria', -5]
Stats before game: [13.270210087691487, 55.32439170406916], [28.86037072079319, 11.617196294893983]
Stats after game:  [18.231472857286523, 34.93340569394821], [28.161779968763323, 9.067339620174803]

Result game 74:   ['Bologna', 'Frosinone', -4]
Stats before game: [25.111249011018106, 8.154942284019953], [18.231472857286523, 34.9334056939

Stats after game:  [25.070940284109184, 5.919824804575643], [21.212683014650324, 11.796500411052037]

Result game 104:   ['Roma', 'Cagliari', 3]
Stats before game: [29.448247779196063, 16.94955439012037], [21.83604437558004, 22.146193808527464]
Stats after game:  [29.948208169436136, 13.321768548235397], [20.057084428850263, 25.05660770383373]

Result game 105:   ['Sassuolo', 'Frosinone', 0]
Stats before game: [28.87778221049541, 8.363788480780537], [27.923733079869404, 15.57410206459745]
Game ignored due to tie
Stats after game:  [28.87778221049541, 8.363788480780537], [27.923733079869404, 15.57410206459745]

Result game 106:   ['Cagliari', 'Juventus', -2]
Stats before game: [20.057084428850263, 25.05660770383373], [77.41708179564961, 49.854162350189966]
Stats after game:  [19.058362042689154, 22.963878461557965], [79.76699191308852, 44.362981819320545]

Result game 107:   ['Milan', 'Genoa', 1]
Stats before game: [39.864333181784446, 27.129922489259435], [14.468865293594053, 23.100522

Stats after game:  [24.179323413626733, 7.840048590857968], [19.6636091959865, 9.096531249269015]

Result game 139:   ['Lazio', 'Chievo', -1]
Stats before game: [26.96631848616002, 12.132690760372967], [16.062993366045536, 8.778841892939175]
Stats after game:  [20.131706070420865, 10.038681230467684], [21.834204001153857, 6.910635578165644]

Result game 140:   ['Lazio', 'Fiorentina', 1]
Stats before game: [20.131706070420865, 10.038681230467684], [19.6636091959865, 9.096531249269015]
Stats after game:  [22.191635965777763, 6.876262580743832], [17.898643125215354, 9.41247258644614]

Result game 141:   ['Inter', 'Cagliari', 2]
Stats before game: [27.625138823249618, 5.912725048919035], [24.179323413626733, 7.840048590857968]
Stats after game:  [28.58872339929213, 5.35240667072134], [23.06740082839915, 7.022862285489993]

Result game 142:   ['Frosinone', 'Napoli', -2]
Stats before game: [20.154048066165107, 8.566591168601402], [29.611986300322933, 4.678126639576564]
Stats after game:  [19

...

Stats after game:  [29.534152044188275, 12.578431368045587], [20.80346409272237, 7.043447351416257]

Result game 239:   ['Napoli', 'Sampdoria', 3]
Stats before game: [31.65458460740112, 2.502711800631983], [24.220434418895003, 1.2559054986164033]
Stats after game:  [31.94890119225874, 3.4336677187492275], [24.165233334851543, 1.3768679240360173]

Result game 240:   ['Lazio', 'Cagliari', 2]
Stats before game: [25.153717658878673, 2.203818042181086], [17.23999337687206, 5.97841831714835]
Stats after game:  [25.33959058234742, 3.306320265955652], [16.987009378769347, 7.614467313982524]

Result game 241:   ['Chievo', 'Cagliari', -3]
Stats before game: [20.80346409272237, 7.043447351416257], [16.987009378769347, 7.614467313982524]
Stats after game:  [17.75840423056147, 5.5437665593296455], [20.340470658545687, 8.399194345796092]

Result game 242:   ['Empoli', 'Milan', 0]
Stats before game: [25.88910509209195, 1.5432995883266987], [27.084812864130186, 4.8831745754964375]
Game ignored due to 

Stats after game:  [28.076367061236773, 3.0572618475314424], [23.923658027125708, 2.8291223655985593]

Result game 272:   ['Genoa', 'Empoli', 1]
Stats before game: [26.723805420662906, 3.271706470577759], [27.28678870785172, 1.6157961766861708]
Stats after game:  [28.722731034056178, 2.437046515730973], [26.300360461066, 1.3253321488729701]

Result game 273:   ['Sampdoria', 'Roma', -1]
Stats before game: [24.165233334851543, 1.3768679240360173], [27.49560396321062, 2.8743034975378605]
Stats after game:  [23.841624838144767, 1.3191149849841082], [28.02841273569152, 4.034561774393179]

Result game 274:   ['Empoli', 'Lazio', -1]
Stats before game: [26.300360461066, 1.3253321488729701], [22.609244733630163, 2.8089406944207815]
Stats after game:  [25.178644002665884, 1.1351468912915628], [25.167066936062252, 2.1171137462808254]

Result game 275:   ['Cagliari', 'Parma', 1]
Stats before game: [18.93950121244748, 9.648587197974642], [25.615686716116407, 4.897829774433423]
Stats after game:  [2

Stats after game:  [27.77456976272049, 4.583045204815399], [22.613986219373892, 3.844822918567433]

Result game 307:   ['Torino', 'Atalanta', 2]
Stats before game: [32.194696231385436, 17.940823088853207], [26.612530117893655, 2.0971356827824463]
Stats after game:  [35.40410932351982, 19.7829705602936], [26.14435107957953, 2.0049416515395784]

Result game 308:   ['Sampdoria', 'Napoli', 3]
Stats before game: [25.669707440216712, 0.7991628825060269], [33.01679131709943, 4.898422091580287]
Stats after game:  [26.488236517201052, 0.5611841647590292], [27.865892375963654, 2.657493638228713]

Result game 309:   ['Atalanta', 'Udinese', 2]
Stats before game: [26.14435107957953, 2.0049416515395784], [27.77456976272049, 4.583045204815399]
Stats after game:  [27.04474612003841, 2.3615484123845554], [25.610333611376927, 3.340668450539833]

Result game 310:   ['Udinese', 'Sampdoria', 1]
Stats before game: [25.610333611376927, 3.340668450539833], [26.488236517201052, 0.5611841647590292]
Stats after 

Stats after game:  [25.763225887103705, 1.316880433566448], [26.423671005728334, 2.585193595246141]

Result game 341:   ['Frosinone', 'Milan', 0]
Stats before game: [24.17772017671997, 5.127486169105092], [23.535451705506613, 2.3163854304548277]
Game ignored due to tie
Stats after game:  [24.17772017671997, 5.127486169105092], [23.535451705506613, 2.3163854304548277]

Result game 342:   ['Bologna', 'Lazio', -2]
Stats before game: [24.850861057251542, 2.7075781420465654], [28.788626184418003, 1.8806412698400725]
Stats after game:  [23.848623737485035, 3.4950217752346204], [29.451985386380112, 1.616974417830641]

Result game 343:   ['Parma', 'Juventus', -1]
Stats before game: [26.423671005728334, 2.585193595246141], [25.451342171836377, 4.600164384176559]
Stats after game:  [25.55621834484776, 2.7236946668605126], [27.58862071258939, 3.737538141624849]

Result game 344:   ['Sampdoria', 'Parma', 2]
Stats before game: [25.861560556455938, 0.5707113225697864], [25.55621834484776, 2.72369466

Stats after game:  [24.896762387808312, 2.5410175925506824], [24.847782454351403, 0.998178543707439]

Result game 376:   ['Sampdoria', 'Empoli', -1]
Stats before game: [26.06132013336628, 0.7421637785008501], [25.16172698734551, 0.36945770969387887]
Stats after game:  [25.286841795212965, 0.9082284941842153], [25.410019707159666, 0.3897111454742619]

Result game 377:   ['Roma', 'Chievo', 0]
Stats before game: [29.410391728159883, 2.1944642845520974], [13.495001129375986, 28.830853106429455]
Game ignored due to tie
Stats after game:  [29.410391728159883, 2.1944642845520974], [13.495001129375986, 28.830853106429455]

Result game 378:   ['Cagliari', 'Torino', 0]
Stats before game: [24.81147149025038, 2.7369395831509475], [24.896762387808312, 2.5410175925506824]
Game ignored due to tie
Stats after game:  [24.81147149025038, 2.7369395831509475], [24.896762387808312, 2.5410175925506824]

Result game 379:   ['Napoli', 'Atalanta', -1]
Stats before game: [29.87700490950855, 1.229695149162633], 

In [37]:
def predict_winner(team1_string,team2_string,stats_dictionary):
    diff = stats_dictionary[team1_string][0] - stats_dictionary[team2_string][0] 
    if diff > 0:
        return 1
    if diff < 0:
        return -1
    else:
        return 0

In [54]:
def calc_prediction_performance(predict_winner, result_list, stats_dictionary):
    correct_predictions = 0
    nr_of_draws = 0
    for match in result_list:
        team1_string = match[0]
        team2_string = match[1]
        true_outcome = match[2]
        predicted_outcome = predict_winner(team1_string, team2_string, stats_dictionary)
        print("Pred outcome: ", predicted_outcome)
        print("True outcome: ", true_outcome)
        if(true_outcome == 0):
            nr_of_draws += 1
            print("Draws: ", nr_of_draws)
        else:
            if np.sign(true_outcome) == np.sign(predicted_outcome):
                correct_predictions += 1
                print("Correct pred: ", correct_predictions)
                       
    print()
    print("correct_predictions ", correct_predictions)
    print("reslist: ", len(result_list))
    print("draws: ", nr_of_draws)
    return correct_predictions / (len(result_list)-nr_of_draws)

In [55]:
calc_prediction_performance(predict_winner, result_list, stats_dictionary)

Pred outcome:  -1
True outcome:  0
Draws:  1
Pred outcome:  -1
True outcome:  0
Draws:  2
Pred outcome:  1
True outcome:  1
Correct pred:  1
Pred outcome:  -1
True outcome:  1
Pred outcome:  -1
True outcome:  -1
Correct pred:  2
Pred outcome:  1
True outcome:  0
Draws:  3
Pred outcome:  1
True outcome:  2
Correct pred:  3
Pred outcome:  -1
True outcome:  0
Draws:  4
Pred outcome:  -1
True outcome:  -3
Correct pred:  4
Pred outcome:  1
True outcome:  2
Correct pred:  5
Pred outcome:  1
True outcome:  4
Correct pred:  6
Pred outcome:  -1
True outcome:  0
Draws:  5
Pred outcome:  -1
True outcome:  0
Draws:  6
Pred outcome:  -1
True outcome:  -1
Correct pred:  7
Pred outcome:  1
True outcome:  -2
Pred outcome:  1
True outcome:  -2
Pred outcome:  1
True outcome:  1
Correct pred:  8
Pred outcome:  -1
True outcome:  0
Draws:  7
Pred outcome:  1
True outcome:  -1
Pred outcome:  -1
True outcome:  0
Draws:  8
Pred outcome:  1
True outcome:  2
Correct pred:  9
Pred outcome:  1
True outcome:  3
Co

Pred outcome:  1
True outcome:  -1
Pred outcome:  1
True outcome:  0
Draws:  87
Pred outcome:  1
True outcome:  -1
Pred outcome:  1
True outcome:  -1
Pred outcome:  1
True outcome:  1
Correct pred:  127
Pred outcome:  -1
True outcome:  2
Pred outcome:  1
True outcome:  1
Correct pred:  128
Pred outcome:  -1
True outcome:  1
Pred outcome:  -1
True outcome:  -4
Correct pred:  129
Pred outcome:  1
True outcome:  0
Draws:  88
Pred outcome:  -1
True outcome:  -1
Correct pred:  130
Pred outcome:  1
True outcome:  1
Correct pred:  131
Pred outcome:  -1
True outcome:  -1
Correct pred:  132
Pred outcome:  1
True outcome:  3
Correct pred:  133
Pred outcome:  -1
True outcome:  0
Draws:  89
Pred outcome:  1
True outcome:  2
Correct pred:  134
Pred outcome:  1
True outcome:  0
Draws:  90
Pred outcome:  -1
True outcome:  -1
Correct pred:  135
Pred outcome:  -1
True outcome:  -1
Correct pred:  136
Pred outcome:  -1
True outcome:  1
Pred outcome:  1
True outcome:  1
Correct pred:  137
Pred outcome:  -

0.6654411764705882